In [1]:
import pandas

from util import add_extra_rows

tableurl = 'data/adult.data'

table = pandas.read_table(tableurl, sep=',', skipinitialspace=True)

table.shape

(32561, 15)

In [2]:
table.iloc[:5, :7]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty


In [3]:
table.iloc[:5, 7:]

,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Husband,White,Male,0,0,13,United-States,<=50K
2,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Husband,Black,Male,0,0,40,United-States,<=50K
4,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
null = table.isnull().sum()

null.sum()

0

In [5]:
from sklearn.model_selection import train_test_split

test_size = 0.2

train, test = train_test_split(table, test_size=test_size, shuffle=False)

In [6]:
sample = add_extra_rows(test)

assert sample.shape[0] == test.shape[0] + 4, 'Fail !'

In [7]:
assert train.isnull().sum().sum() == test.isnull().sum().sum(), 'Fail !'

In [8]:
from tensorflow_data_validation import generate_statistics_from_dataframe

from tensorflow_data_validation import visualize_statistics

train_stats = generate_statistics_from_dataframe(train)

# visualize_statistics(train_stats)

In [9]:
test_stats = generate_statistics_from_dataframe(test)

# visualize_statistics(test)

In [10]:
sample_stats = generate_statistics_from_dataframe(sample)

# visualize_statistics(sample_stats)

In [11]:
statlist = {}

statlist['lhs_statistics'] = train_stats; statlist['lhs_name'] = 'Train'

statlist['rhs_statistics'] = test_stats; statlist['rhs_name'] = 'Test'

# visualize_statistics(**statlist)

In [12]:
statlist = {}

statlist['lhs_statistics'] = sample_stats; statlist['lhs_name'] = 'Sample'

statlist['rhs_statistics'] = test_stats; statlist['rhs_name'] = 'Test'

# visualize_statistics(**statlist)

In [13]:
lower = sample['age'] > 16

upper = sample['age'] < 91

sample = sample[lower & upper]

sample = sample.dropna()

[sample.age.min(), sample.age.max()]

[17, 90]

In [14]:
sample_stats = generate_statistics_from_dataframe(sample)

statlist['lhs_statistics'] = sample_stats; statlist['lhs_name'] = 'Sample'

statlist['rhs_statistics'] = test_stats; statlist['rhs_name'] = 'Test'

# visualize_statistics(**statlist)

---

In [15]:
import tensorflow_data_validation as tfdv

schemas = tfdv.infer_schema(train_stats)

# tfdv.display_schema(schemas)

---

In [16]:
anomalies = tfdv.validate_statistics(test_stats, schemas)

len(anomalies.anomaly_info.items())

0

In [17]:
anomalies = tfdv.validate_statistics(sample_stats, schemas)

len(anomalies.anomaly_info.items())

3

In [18]:
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'occupation',Unexpected string values,Examples contain values missing from the schema: Gamer (<1%).
'race',Unexpected string values,Examples contain values missing from the schema: Asian (<1%).
'native-country',Unexpected string values,Examples contain values missing from the schema: Mongolia (<1%).
